In [1]:
from svm_gpu.svm import SVM 

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from matplotlib import pyplot as plt

import torch
import torch.nn as nn
from pytorch_tabnet.tab_model import TabNetClassifier


In [4]:
path = '/Data2/folder_ti/data/'
sample_csv = 'sample_train.csv'

In [5]:
train = pd.read_csv(path+sample_csv)


In [6]:
train_shuf=train.sample(frac=1).reset_index(drop=True)

In [7]:
train_shuf.head()

,bank_id,product_id,loan_limit,loan_rate,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,...,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,y,Set,sample
0,38,134,12000000.0,7.3,1969.0,1.0,640.0,28000000.0,EARNEDINCOME2,202103.0,...,기타가족소유,15000000.0,전월세보증금,NaN,NaN,1.0,0.0,0.0,train,1
1,36,157,36000000.0,11.5,1980.0,1.0,890.0,36000000.0,EARNEDINCOME,201402.0,...,자가,50000000.0,대환대출,0.0,NaN,6.0,22000000.0,0.0,train,1
2,38,223,10000000.0,13.3,1979.0,1.0,NaN,30000000.0,EARNEDINCOME,202105.0,...,자가,5000000.0,생활비,NaN,NaN,5.0,6000000.0,0.0,valid,1
3,58,175,35000000.0,13.8,1990.0,1.0,840.0,35000000.0,EARNEDINCOME,202110.0,...,전월세,100000000.0,대환대출,0.0,NaN,3.0,27000000.0,0.0,train,1
4,21,196,7000000.0,6.1,1977.0,1.0,1000.0,68000000.0,EARNEDINCOME,201903.0,...,자가,50000000.0,생활비,NaN,NaN,1.0,50000000.0,0.0,train,1


In [8]:
train_shuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514428 entries, 0 to 514427
Data columns (total 21 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   bank_id                              514428 non-null  int64  
 1   product_id                           514428 non-null  int64  
 2   loan_limit                           514158 non-null  float64
 3   loan_rate                            514158 non-null  float64
 4   birth_year                           509853 non-null  float64
 5   gender                               509853 non-null  float64
 6   credit_score                         451706 non-null  float64
 7   yearly_income                        514428 non-null  float64
 8   income_type                          514428 non-null  object 
 9   company_enter_month                  499030 non-null  float64
 10  employment_type                      514428 non-null  object 
 11  houseown_type

In [9]:
y = list(set(train['y'].tolist()))
print(y)

[0.0, 1.0]


In [10]:
a = list(set(train['Set'].tolist()))
print(a)

['test', 'train', 'valid']


In [11]:
categorical = ['income_type','employment_type','houseown_type','purpose']
equal = {'SWITCHLOAN' :'대환대출','ETC':'기타','HOUSEDEPOSIT':'전월세보증금','BUSINESS':'사업자금','LIVING':'생활비','BUYHOUSE':'주택구입',
        'BUYCAR':'자동차구입','INVEST':'투자'}
cat_trans = dict()
for cat in categorical :
    cat_trans[cat] = dict()
    if cat == 'purpose':
        type_ = list(set(train[cat].tolist()))
        for i, ty in enumerate(type_) :
            if ty in equal.keys():
                type_[i] = equal[ty]
        type_ = list(set(type_))
        for i, ty in enumerate(type_) :
            cat_trans[cat][ty] = i
    else :
        type_ = list(set(train[cat].tolist()))
        for i, ty in enumerate(type_) :
            cat_trans[cat][ty] = i


In [12]:
cat_trans

{'income_type': {'EARNEDINCOME2': 0,
  'EARNEDINCOME': 1,
  'FREELANCER': 2,
  'PRIVATEBUSINESS': 3,
  'PRACTITIONER': 4,
  'OTHERINCOME': 5},
 'employment_type': {'정규직': 0, '기타': 1, '계약직': 2, '일용직': 3},
 'houseown_type': {'자가': 0, '기타가족소유': 1, '배우자': 2, '전월세': 3},
 'purpose': {'사업자금': 0,
  '생활비': 1,
  '주택구입': 2,
  '기타': 3,
  '자동차구입': 4,
  '전월세보증금': 5,
  '투자': 6,
  '대환대출': 7}}

In [13]:
cat_trans['income_type'].keys()

dict_keys(['EARNEDINCOME2', 'EARNEDINCOME', 'FREELANCER', 'PRIVATEBUSINESS', 'PRACTITIONER', 'OTHERINCOME'])

In [14]:
from tqdm import tqdm
for cat in categorical :
    if cat == 'purpose' :
        for j in equal.keys():
            train[cat][train[cat]==j] = cat_trans[cat][equal[j]]
        for k in tqdm(cat_trans[cat].keys()) :
            train[cat][train[cat]==k] = cat_trans[cat][k]
    else :
        for k in tqdm(cat_trans[cat].keys()) :
            train[cat][train[cat]==k] = cat_trans[cat][k]        

  0%|                                                                                                                       | 0/6 [00:00<?, ?it/s]/tmp/ipykernel_30333/4092950377.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==k] = cat_trans[cat][k]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 24.98it/s]
/tmp/ipykernel_30333/4092950377.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat][train[cat]==j] = cat_trans[cat][equal[j]]
  0%|                                                            

In [15]:
train_shuf.head()

,bank_id,product_id,loan_limit,loan_rate,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,...,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,y,Set,sample
0,38,134,12000000.0,7.3,1969.0,1.0,640.0,28000000.0,EARNEDINCOME2,202103.0,...,기타가족소유,15000000.0,전월세보증금,NaN,NaN,1.0,0.0,0.0,train,1
1,36,157,36000000.0,11.5,1980.0,1.0,890.0,36000000.0,EARNEDINCOME,201402.0,...,자가,50000000.0,대환대출,0.0,NaN,6.0,22000000.0,0.0,train,1
2,38,223,10000000.0,13.3,1979.0,1.0,NaN,30000000.0,EARNEDINCOME,202105.0,...,자가,5000000.0,생활비,NaN,NaN,5.0,6000000.0,0.0,valid,1
3,58,175,35000000.0,13.8,1990.0,1.0,840.0,35000000.0,EARNEDINCOME,202110.0,...,전월세,100000000.0,대환대출,0.0,NaN,3.0,27000000.0,0.0,train,1
4,21,196,7000000.0,6.1,1977.0,1.0,1000.0,68000000.0,EARNEDINCOME,201903.0,...,자가,50000000.0,생활비,NaN,NaN,1.0,50000000.0,0.0,train,1


In [16]:
train_shuf.nunique()

bank_id                                 61
product_id                             168
loan_limit                             203
loan_rate                              173
birth_year                              72
gender                                   2
credit_score                            76
yearly_income                          456
income_type                              6
company_enter_month                    481
employment_type                          4
houseown_type                            4
desired_amount                         437
purpose                                 16
personal_rehabilitation_yn               2
personal_rehabilitation_complete_yn      2
existing_loan_cnt                       74
existing_loan_amt                      853
y                                        2
Set                                      3
sample                                   1
dtype: int64

In [17]:
train_indices = train_shuf[train_shuf.Set=="train"].index
valid_indices = train_shuf[train_shuf.Set=="valid"].index
test_indices = train_shuf[train_shuf.Set=="test"].index


In [18]:
for col in train.columns :
    print('col name & nan : ', col,'/',train[col].isnull().sum())

col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 270
col name & nan :  loan_rate / 270
col name & nan :  birth_year / 4575
col name & nan :  gender / 4575
col name & nan :  credit_score / 62722
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_enter_month / 15398
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  purpose / 0
col name & nan :  personal_rehabilitation_yn / 294183
col name & nan :  personal_rehabilitation_complete_yn / 462315
col name & nan :  existing_loan_cnt / 107754
col name & nan :  existing_loan_amt / 152940
col name & nan :  y / 0
col name & nan :  Set / 0
col name & nan :  sample / 0


In [21]:
train.head()

,bank_id,product_id,loan_limit,loan_rate,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,...,houseown_type,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,y,Set,sample
0,52,187,3000000.0,6.2,1983.0,1.0,740.0,39000000.0,1,201203.0,...,3,80000000.0,7,0.0,NaN,2.0,58000000.0,0.0,train,1
1,38,223,18000000.0,13.3,1983.0,1.0,740.0,39000000.0,1,201203.0,...,3,80000000.0,7,0.0,NaN,2.0,58000000.0,0.0,train,1
2,49,39,20000000.0,14.8,1983.0,1.0,740.0,39000000.0,1,201203.0,...,3,80000000.0,7,0.0,NaN,2.0,58000000.0,0.0,test,1
3,10,65,10000000.0,15.6,1998.0,0.0,630.0,28000000.0,1,202110.0,...,3,10000000.0,1,0.0,NaN,3.0,36000000.0,0.0,valid,1
4,32,257,12000000.0,7.2,1998.0,0.0,630.0,28000000.0,1,202110.0,...,3,10000000.0,1,0.0,NaN,3.0,36000000.0,0.0,train,1


In [19]:
nunique = train_shuf.nunique()
types = train.dtypes
target = 'y'
categorical_columns = ['bank_id','product_id','gender','income_type','employment_type','houseown_type'
                       ,'purpose','','personal_rehabilitation_yn','personal_rehabilitation_complete_yn']
train_shuf = train_shuf.astype({'bank_id':'object','product_id':'object','gender':'object','personal_rehabilitation_yn':'object','personal_rehabilitation_complete_yn':'object'})
categorical_dims =  {}
for col in train_shuf.columns:
    if types[col] == 'object' or col in categorical_columns :
        if col == 'Set' :
            continue
        print(col, train_shuf[col].nunique())
        l_enc = LabelEncoder()
        if col == 'gender' :
            train_shuf[col] = train_shuf[col].fillna(np.random.choice([0,1], p =[.5, .5]).item())
        else :
            train_shuf[col] = train_shuf[col].fillna(-1)
        train_shuf[col] = l_enc.fit_transform(train_shuf[col].values)
#        categorical_columns.append(col)
        categorical_dims[col] = len(l_enc.classes_)
    else:
        train_shuf.fillna(train_shuf.loc[:, col].mean(), inplace=True)


# Categorical Embedding을 위해 Categorical 변수의 차원과 idxs를 담음.
unused_feat = ['Set']
features = [ col for col in train_shuf.columns if col not in unused_feat+[target]] 
cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]
cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]
#train_shuf = train_shuf.astype({'income_type':'int64','employment_type':'int64','houseown_type':'int64','purpose':'int64'})

bank_id 61
product_id 168
gender 3
income_type 6
employment_type 4
houseown_type 4
purpose 16
personal_rehabilitation_yn 3
personal_rehabilitation_complete_yn 3


In [20]:
y = list(set(train_shuf['y'].tolist()))
print(y)

[0.0, 1.0]


In [20]:
train_shuf['company_work_year'] = train_shuf['company_enter_month'].apply(lambda x : 2023 - round(x,-2)/100)
print('d')
train_shuf['company_enter'] = train_shuf['company_enter_month'].apply(lambda x : round(x,-2)/100)
print('d')

d
d


In [21]:
#train_shuf = train_shuf.drop(['company_enter_month'],axis = 1)
train_shuf = train_shuf.drop(['sample'],axis = 1)

In [22]:
train_shuf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514428 entries, 0 to 514427
Data columns (total 22 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   bank_id                              514428 non-null  int64  
 1   product_id                           514428 non-null  int64  
 2   loan_limit                           514428 non-null  float64
 3   loan_rate                            514428 non-null  float64
 4   birth_year                           514428 non-null  float64
 5   gender                               514428 non-null  int64  
 6   credit_score                         514428 non-null  float64
 7   yearly_income                        514428 non-null  float64
 8   income_type                          514428 non-null  int64  
 9   company_enter_month                  514428 non-null  float64
 10  employment_type                      514428 non-null  int64  
 11  houseown_type

In [23]:
train_shuf.head()

,bank_id,product_id,loan_limit,loan_rate,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,...,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,y,Set,company_work_year,company_enter
0,59,134,20000000.0,16.0,2000.0,1,6.500000e+02,33000000.0,0,202203.0,...,5000000.0,11,2,2,2.265132e+07,2.265132e+07,0.0,train,1.0,2022.0
1,56,109,50000000.0,5.0,1963.0,1,2.265132e+07,50000000.0,0,202105.0,...,100000000.0,11,2,2,2.265132e+07,2.265132e+07,0.0,train,2.0,2021.0
2,35,125,35000000.0,19.9,1986.0,1,2.265132e+07,44000000.0,0,201401.0,...,40000000.0,14,0,0,2.265132e+07,2.265132e+07,0.0,train,9.0,2014.0
3,5,19,17000000.0,16.7,1980.0,1,6.700000e+02,40000000.0,0,202112.0,...,20000000.0,9,2,2,5.000000e+00,7.000000e+07,0.0,train,2.0,2021.0
4,25,106,27000000.0,12.9,1968.0,1,7.500000e+02,30000000.0,1,202002.0,...,23000000.0,9,2,2,2.000000e+00,2.700000e+07,0.0,train,3.0,2020.0


In [24]:
#train.to_csv('train_basic.csv',index=False)

In [25]:
for col in train_shuf.columns :
    print('col name & nan : ', col,'/',train_shuf[col].isnull().sum())

col name & nan :  bank_id / 0
col name & nan :  product_id / 0
col name & nan :  loan_limit / 0
col name & nan :  loan_rate / 0
col name & nan :  birth_year / 0
col name & nan :  gender / 0
col name & nan :  credit_score / 0
col name & nan :  yearly_income / 0
col name & nan :  income_type / 0
col name & nan :  company_enter_month / 0
col name & nan :  employment_type / 0
col name & nan :  houseown_type / 0
col name & nan :  desired_amount / 0
col name & nan :  purpose / 0
col name & nan :  personal_rehabilitation_yn / 0
col name & nan :  personal_rehabilitation_complete_yn / 0
col name & nan :  existing_loan_cnt / 0
col name & nan :  existing_loan_amt / 0
col name & nan :  y / 0
col name & nan :  Set / 0
col name & nan :  company_work_year / 0
col name & nan :  company_enter / 0


In [26]:
features = [ col for col in train_shuf.columns if col not in unused_feat+[target]]

In [27]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514428 entries, 0 to 514427
Data columns (total 21 columns):
 #   Column                               Non-Null Count   Dtype  
---  ------                               --------------   -----  
 0   bank_id                              514428 non-null  int64  
 1   product_id                           514428 non-null  int64  
 2   loan_limit                           514158 non-null  float64
 3   loan_rate                            514158 non-null  float64
 4   birth_year                           509853 non-null  float64
 5   gender                               509853 non-null  float64
 6   credit_score                         451706 non-null  float64
 7   yearly_income                        514428 non-null  float64
 8   income_type                          514428 non-null  object 
 9   company_enter_month                  499030 non-null  float64
 10  employment_type                      514428 non-null  object 
 11  houseown_type

In [37]:
X_train = train_shuf[features].values[train_indices]
y_train = train_shuf[target].values[train_indices]

X_valid = train_shuf[features].values[valid_indices]
y_valid = train_shuf[target].values[valid_indices]

X_test = train_shuf[features].values[test_indices]
y_test = train_shuf[target].values[test_indices]

In [29]:
print(X_train.shape)
print(y_train.shape)
print(X_valid.shape)
print(y_valid.shape)
print(X_test.shape)
print(y_test.shape)

(411288, 20)
(411288,)
(51609, 20)
(51609,)
(51531, 20)
(51531,)


In [30]:
import torch
DEVICE = 'cuda:4' if torch.cuda.is_available() else 'cpu'
print("Using {}".format(DEVICE))

Using cuda:4


In [31]:
#train = train.astype({'bank_id':'object','product_id':'object','gender':'object','income_type':'object',
#                     'employment_type':'object','houseown_type':'object','purpose':'object','personal_rehabilitation_yn':'object',
#                     'personal_rehabilitation_complete_yn':'object'})

In [32]:
train_shuf.head()

,bank_id,product_id,loan_limit,loan_rate,birth_year,gender,credit_score,yearly_income,income_type,company_enter_month,...,desired_amount,purpose,personal_rehabilitation_yn,personal_rehabilitation_complete_yn,existing_loan_cnt,existing_loan_amt,y,Set,company_work_year,company_enter
0,59,134,20000000.0,16.0,2000.0,1,6.500000e+02,33000000.0,0,202203.0,...,5000000.0,11,2,2,2.265132e+07,2.265132e+07,0.0,train,1.0,2022.0
1,56,109,50000000.0,5.0,1963.0,1,2.265132e+07,50000000.0,0,202105.0,...,100000000.0,11,2,2,2.265132e+07,2.265132e+07,0.0,train,2.0,2021.0
2,35,125,35000000.0,19.9,1986.0,1,2.265132e+07,44000000.0,0,201401.0,...,40000000.0,14,0,0,2.265132e+07,2.265132e+07,0.0,train,9.0,2014.0
3,5,19,17000000.0,16.7,1980.0,1,6.700000e+02,40000000.0,0,202112.0,...,20000000.0,9,2,2,5.000000e+00,7.000000e+07,0.0,train,2.0,2021.0
4,25,106,27000000.0,12.9,1968.0,1,7.500000e+02,30000000.0,1,202002.0,...,23000000.0,9,2,2,2.000000e+00,2.700000e+07,0.0,train,3.0,2020.0


In [33]:
#cat_idxs.pop(cat_idxs.index(14))

In [34]:
#cat_idxs.append(12)

In [ ]:
#cat_idxs

# TabNet

In [ ]:
from sklearn.metrics import f1_score ,roc_auc_score
import time

In [ ]:
clf = TabNetClassifier(n_d=64, n_a=64, n_steps=5,
                       gamma=1.5, n_independent=2, n_shared=2,
                       cat_emb_dim=1, lambda_sparse=1e-4, 
                       momentum=0.3, clip_value=2., optimizer_fn=torch.optim.Adam,
                       optimizer_params=dict(lr=2e-2), scheduler_params = {"gamma": 0.97,
                         "step_size": 20},
                       scheduler_fn=torch.optim.lr_scheduler.StepLR, epsilon=1e-15,
                       device_name = DEVICE
)
s = time.time()

clf.fit(X_train=X_train, y_train=y_train,eval_set=[(X_valid, y_valid)],
    max_epochs=30 , patience=20,
    batch_size=10240, virtual_batch_size=1280,
    num_workers=0,
    weights=1,
    drop_last=False
)

print('TabNet gpu sec : ',time.time()- s)

pred_valid = clf.predict(X_valid)
pred_test = clf.predict(X_test)


f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
f1_test = f1_score(y_test, pred_test)
print("Valid F1 score: {}".format(f1_valid))
print("Test F1 score: {}".format(f1_test))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
roc_auc_t = roc_auc_score(y_test, pred_test) 
print("Valid ROC AUC Score: {}".format(roc_auc))
print("Test ROC AUC Score: {}".format(roc_auc_t))


# XGBoost

In [ ]:
from xgboost import XGBClassifier
import time
xgb = XGBClassifier(n_estimators = 500, learning_rate = 0.1,max_depth = 100)
s = time.time()
xgb.fit(X_train,y_train)
print('second : ',round(time.time()-s,2))


In [2]:
xgb = XGBClassifier(n_estimators = 500, learning_rate = 0.1,max_depth = 100,tree_method="gpu_hist")

s = time.time()
xgb.fit(X_train,y_train)
print('second : ',round(time.time()-s,2))


NameError: name 'X_train' is not defined

In [37]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import fetch_california_housing

# Fetch dataset using sklearn
data = fetch_california_housing()
print( data.DESCR)
X = data.data
y = data.target

num_round = 500

param = {
    "eta": 0.05,
    "max_depth": 10,
    "tree_method": "gpu_hist",
}

# GPU accelerated training
dtrain = xgb.DMatrix(X, label=y, feature_names=data.feature_names)
%time model = xgb.train(param, dtrain,num_round)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [38]:
# GPU accelerated training
dtrain = xgb.DMatrix(X_train, label=y_train)#, feature_names=data.feature_names)
%time model = xgb.train(param, dtrain,num_round)

CPU times: user 12.8 s, sys: 2.4 s, total: 15.3 s
Wall time: 13.2 s


In [40]:
%%time
# Compute shap interaction values using GPU
dvalid = xgb.DMatrix(X_valid, label = y_valid)
shap_interaction_values = model.predict(dvalid, pred_interactions=True)

CPU times: user 1min 7s, sys: 45.8 s, total: 1min 53s
Wall time: 1min 52s


In [41]:
shap_interaction_values

array([[[ 2.6279433e-02,  3.3719591e-03,  2.8574996e-04, ...,
         -7.5724995e-05, -6.2288615e-07,  0.0000000e+00],
        [ 3.3719593e-03,  4.4005448e-03, -4.2635979e-04, ...,
          7.2990268e-05,  1.4417850e-07,  0.0000000e+00],
        [ 2.8574993e-04, -4.2635977e-04,  2.7599300e-03, ...,
         -4.3667624e-06, -3.8315736e-08,  0.0000000e+00],
        ...,
        [-7.5724958e-05,  7.2990275e-05, -4.3667787e-06, ...,
          1.1273567e-04,  2.3825626e-07,  0.0000000e+00],
        [-6.2288632e-07,  1.4417407e-07, -3.8315953e-08, ...,
          2.3825623e-07, -1.4023786e-06,  0.0000000e+00],
        [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
          0.0000000e+00,  0.0000000e+00,  5.4807156e-02]],

       [[-1.1029463e-02, -1.7591909e-04, -1.7434716e-03, ...,
         -1.1704201e-05,  3.7865240e-07,  0.0000000e+00],
        [-1.7591918e-04,  1.3516568e-04, -1.1603360e-03, ...,
          3.5617613e-05, -1.5708606e-07,  0.0000000e+00],
        [-1.7434716e-03, 

In [46]:
import xgboost as xgb
model = xgb.XGBClassifier(n_estimators=400, learning_rate=0.05, gamma=0, subsample=0.75, colsample_bytree=1,
                            max_depth=7, tree_method='gpu_hist',gpu_id=4)
s = time.time()
model.fit(X_train,y_train)
print('gpu_second : ',time.time()-s)

gpu_second :  2.405137538909912


In [48]:
pred_valid = model.predict(X_valid)
pred_test = model.predict(X_test)
from sklearn.metrics import f1_score ,roc_auc_score
f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
f1_test = f1_score(y_test, pred_test)
print("Valid F1 score: {}".format(f1_valid))
print("Test F1 score: {}".format(f1_test))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
roc_auc_t = roc_auc_score(y_test, pred_test) 
print("Valid ROC AUC Score: {}".format(roc_auc))
print("Test ROC AUC Score: {}".format(roc_auc_t))

Valid F1 score: 0.13950578667500782
Test F1 score: 0.13701236917221693
Valid ROC AUC Score: 0.5379993529999849
Test ROC AUC Score: 0.5373198000968298


# LightGBM

In [39]:
#!pip install lightgbm

  Using cached lightgbm-3.3.2-py3-none-manylinux1_x86_64.whl (2.0 MB)


In [35]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(device='gpu',gpu_device_id=4,gpu_platform_id=4)

In [36]:
model.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 22569, number of negative: 388719
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1699
[LightGBM] [Info] Number of data points in the train set: 411288, number of used features: 20
[LightGBM] [Info] Using GPU Device: Quadro RTX 8000, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 17 dense feature groups (7.84 MB) transferred to GPU in 0.048596 secs. 1 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.054874 -> initscore=-2.846279
[LightGBM] [Info] Start training from score -2.846279


LGBMClassifier(device='gpu')

In [38]:
pred_valid = model.predict(X_valid, num_iteration=model.best_iteration_)
pred_test = model.predict(X_test, num_iteration=model.best_iteration_)
from sklearn.metrics import f1_score ,roc_auc_score
f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
f1_test = f1_score(y_test, pred_test)
print("Valid F1 score: {}".format(f1_valid))
print("Test F1 score: {}".format(f1_test))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
roc_auc_t = roc_auc_score(y_test, pred_test) 
print("Valid ROC AUC Score: {}".format(roc_auc))
print("Test ROC AUC Score: {}".format(roc_auc_t))

Valid F1 score: 0.09448298645523621
Test F1 score: 0.08760107816711589
Valid ROC AUC Score: 0.5246866275992033
Test ROC AUC Score: 0.522749369680647


In [44]:


# coding: utf-8
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

import lightgbm as lgb

def AUC(y_true, y_pred):
    return 'RMSLE', roc_auc_score(y_true,y_pred), False

print('Starting training...')
# train
gbm = lgb.LGBMClassifier(num_leaves=31,
                        learning_rate=0.05,
                        n_estimators=20)
s = time.time()
gbm.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric=AUC,
        callbacks=[lgb.early_stopping(5)])
print('lgbm cpu sec : ',round(time.time()-s,2))
print('Starting predicting...')


pred_valid = gbm.predict(X_valid, num_iteration=gbm.best_iteration_)
pred_test = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
from sklearn.metrics import f1_score ,roc_auc_score
f1_valid = f1_score(y_valid, pred_valid) # y_true : 정답 값, y_pred : 예측 값 
f1_test = f1_score(y_test, pred_test)
print("Valid F1 score: {}".format(f1_valid))
print("Test F1 score: {}".format(f1_test))
 

from sklearn.metrics import roc_auc_score 
roc_auc = roc_auc_score(y_valid, pred_valid)
roc_auc_t = roc_auc_score(y_test, pred_test) 
print("Valid ROC AUC Score: {}".format(roc_auc))
print("Test ROC AUC Score: {}".format(roc_auc_t))


# feature importances
print(f'Feature importances: {list(gbm.feature_importances_)}')


# self-defined eval metric
# f(y_true: array, y_pred: array) -> name: str, eval_result: float, is_higher_better: bool
# Root Mean Squared Logarithmic Error (RMSLE)
def rmsle(y_true, y_pred):
    return 'RMSLE', np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2))), False


print('Starting training with custom eval function...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=rmsle,
        callbacks=[lgb.early_stopping(5)])


# another self-defined eval metric
# f(y_true: array, y_pred: array) -> name: str, eval_result: float, is_higher_better: bool
# Relative Absolute Error (RAE)
def rae(y_true, y_pred):
    return 'RAE', np.sum(np.abs(y_pred - y_true)) / np.sum(np.abs(np.mean(y_true) - y_true)), False


print('Starting training with multiple custom eval functions...')
# train
gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=[rmsle, rae],
        callbacks=[lgb.early_stopping(5)])

print('Starting predicting...')
# predict
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration_)
# eval
rmsle_test = rmsle(y_test, y_pred)[1]
rae_test = rae(y_test, y_pred)[1]
print(f'The RMSLE of prediction is: {rmsle_test}')
print(f'The RAE of prediction is: {rae_test}')

# other scikit-learn modules
estimator = lgb.LGBMRegressor(num_leaves=31)

param_grid = {
    'learning_rate': [0.01, 0.1, 1],
    'n_estimators': [20, 40]
}

gbm = GridSearchCV(estimator, param_grid, cv=3)
gbm.fit(X_train, y_train)

print(f'Best parameters found by grid search are: {gbm.best_params_}')


Starting training...
Training until validation scores don't improve for 5 rounds
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: 0.207028	valid_0's RMSLE: 0.784352
lgbm cpu sec :  0.39
Starting predicting...
Valid F1 score: 0.0
Test F1 score: 0.0
Valid ROC AUC Score: 0.5
Test ROC AUC Score: 0.5
Feature importances: [1, 2, 2, 13, 0, 0, 4, 0, 3, 1, 0, 0, 3, 0, 0, 0, 0, 1, 0, 0]
Starting training with custom eval function...
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[20]	valid_0's binary_logloss: 0.177081	valid_0's RMSLE: 0.148726
Starting training with multiple custom eval functions...
Training until validation scores don't improve for 5 rounds
Did not meet early stopping. Best iteration is:
[20]	valid_0's binary_logloss: 0.177081	valid_0's RMSLE: 0.148726	valid_0's RAE: 0.954013
Starting predicting...
The RMSLE of prediction is: 0.1603280678842806
The RAE of prediction is: 0.5279017178021235
Best paramete

# SVM

In [42]:
#!pip install cupy

     |████████████████████████████████| 1.8 MB 1.1 MB/s eta 0:00:01
  Using cached fastrlock-0.8-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (49 kB)
  Created wheel for cupy: filename=cupy-11.1.0-cp38-cp38-linux_x86_64.whl size=101205611 sha256=aa4cfa890884e16582d8dd5f43d9f8b04faaf7bcf356ed683ba1530102ef28bc
  Stored in directory: /root/.cache/pip/wheels/ee/33/9b/d653787513325e1a77182ff1931bd2c64a23e6a02b8a465cdb
Successfully built cupy


In [35]:
from svm_gpu.svm import SVM 

In [36]:
import cupy as xp 
import sklearn.model_selection
from sklearn.datasets import load_digits
from svm_gpu.svm import SVM 

# Load the digits dataset, made up of 1797 8x8 images of hand-written digits
digits = load_digits()

# Divide the data into train, test sets 
x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(digits.data, digits.target)

# Move data to GPU
x_train = xp.asarray(x_train)
x_test =  xp.asarray(x_test)
y_train = xp.asarray(y_train)
y_test = xp.asarray(y_test)

# initialize a new predictor
svm = SVM(kernel='rbf', kernel_params={'sigma': 15}, classification_strategy='ovr', x=x_train, y=y_train, n_folds=3, use_optimal_lambda=True, display_plots=True)

svm.fit(X_train, y_train)

# predict things
svm.predict(x_test)

# compute misclassification error
misclassification_error = svm.compute_misclassification_error(x_test, y_test)
print('Misclassification error, lambda = {} : {}\n'.format(svm._lambduh, misclassification_error))

LinAlgError: Internal Error.

In [38]:
svm = SVM(kernel='rbf', kernel_params={'sigma': 15}, classification_strategy='ovr', x=x_train, y=y_train, n_folds=3, use_optimal_lambda=True, display_plots=True)

svm.fit(X_train, y_train)

MemoryError: Unable to allocate 1.23 TiB for an array with shape (411288, 411288) and data type float64

In [39]:
X_train.shape

(411288, 20)